# **Evaluation**

In [1]:
import Discount

In [2]:
DATA_PATH = '/Users/leonhagel/Documents/coding/github/mlim-g2/data/'
EXPORT_PATH = '/Users/leonhagel/Documents/coding/github/mlim-g2/export/'

In [3]:
files = {
    'data': [
        #DATA_PATH + 'baskets.parquet',
        #DATA_PATH + 'coupon_index.parquet', 
        #DATA_PATH + 'coupons.parquet', 
        #DATA_PATH + 'merged.parquet',
        #DATA_PATH + 'clean.parquet', 
        DATA_PATH + 'prepare.parquet'
    ]#, 
    #'mappings': [
        #EXPORT_PATH + 'product_histories.parquet',
        #EXPORT_PATH + 'prices.parquet',
    #]
}

In [4]:
discount_model = Discount.No_Cross_Effects()
discount_model.load(files)

discount_model.pipeline(89, 4, [0.15, 0.2, 0.25, 0.3])
discount_model.top_coupons

[discount] train-log-loss: 0.07631113321309001


,shopper,week,coupon,product,discount
22171,0,89,0,171,0.15
22067,0,89,1,67,0.15
22157,0,89,2,157,0.15
22076,0,89,3,76,0.30
22116,0,89,4,116,0.15
...,...,...,...,...,...
44499810,1999,89,0,60,0.15
44499781,1999,89,1,31,0.20
44499858,1999,89,2,108,0.15
44499791,1999,89,3,41,0.20


In [75]:
class Evaluation:
    def __init__(self, discount_model):
        self.model = discount_model
        self.model_input = {}
    

In [76]:
evaluation = Evaluation(discount_model)

In [77]:
def get_model_inputs(self):
    model = self.model
    top_coupons = model.top_coupons
    
    _, _, X_zero, _ = model.train_test_split(model.test_week, model.train_window, df=model.df, features=model.features)
    X_top_coupons = X_zero.copy()
    X_top_coupons.loc[top_coupons.index, 'discount'] = top_coupons['discount']
    X_top_coupons['discount'] = X_top_coupons['discount'].astype('float32')
    self.model_input['zero'] = X_zero
    self.model_input['top_coupons'] = X_top_coupons
    
setattr(Evaluation, 'get_model_inputs', get_model_inputs)
del get_model_inputs

In [78]:
evaluation.get_model_inputs()

In [79]:
evaluation.model_input['top_coupons']

,price,discount,weeks_since_last_purchase,trend_1,trend_3,trend_5,product_freq,shopper_WOE,product_WOE
22000,688.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.541070
22001,560.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.039796,0.638078
22002,773.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.039796,-0.776683
22003,722.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.039796,-1.401178
22004,620.0,0.0,19.0,0.0,0.0,0.0,0.089888,0.039796,-0.128700
...,...,...,...,...,...,...,...,...,...
44499995,549.0,0.0,19.0,0.0,0.0,0.0,0.123596,0.071599,-0.878618
44499996,702.0,0.3,103.0,0.0,0.0,0.0,0.000000,0.071599,-0.343367
44499997,670.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.071599,-1.559153
44499998,490.0,0.0,103.0,0.0,0.0,0.0,0.000000,0.071599,0.837201


In [80]:
def get_total_expected_revenue(self, model_input='top_coupons'):
    
    if type(model_input) == str:
        model_input = self.model_input[model_input]
    model = self.model
    prices = model.data['prepare'].loc[model_input.index, 'price'].values
    probs = model.predict(model.model, model_input)
    return np.dot(probs, prices)
    
setattr(Evaluation, 'get_total_expected_revenue', get_total_expected_revenue)
del get_total_expected_revenue

In [86]:
evaluation.get_total_expected_revenue('zero')

9057066.70059505

In [87]:
evaluation.get_total_expected_revenue('top_coupons')

10765702.247214857